In [2896]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# **Importing Dataset**

In [2897]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

# **Take look at data**

In [2898]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2899]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [2900]:
train.describe(include="all")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


# **Look at features (extract and engineering)**

In [2901]:
train.groupby("Pclass")["Survived"].mean()

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

In [2902]:
# Extract Title from name
train["Name"].str.split(",", expand=True)[1].str.split(".", expand=True)[0].str.strip().unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [2903]:
train["Title"] = train["Name"].str.split(",", expand=True)[1].str.split(".", expand=True)[0].str.strip()
test["Title"] = test["Name"].str.split(",", expand=True)[1].str.split(".", expand=True)[0].str.strip()

In [2904]:
train["Title"].value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: Title, dtype: int64

In [2905]:
# Encoding function to encode title
def title_encoder(title):
    if title == "Mr":
        return 0
    elif title == "Miss":
        return 1
    elif title == "Mrs":
        return 2
    elif title == "Master":
        return 3
    else:
        return 4

In [2906]:
train["Title"] = train["Title"].apply(title_encoder)
test["Title"] = test["Title"].apply(title_encoder)

In [2907]:
train["Title"].value_counts(), train.groupby("Title")["Survived"].mean()

(0    517
 1    182
 2    125
 3     40
 4     27
 Name: Title, dtype: int64,
 Title
 0    0.156673
 1    0.697802
 2    0.792000
 3    0.575000
 4    0.444444
 Name: Survived, dtype: float64)

In [2908]:
sex_map = {
    "male": 0,
    "female": 1
}
train["Sex"] = train["Sex"].map(sex_map)
test["Sex"] = test["Sex"].map(sex_map)

In [2909]:
train["Sex"].value_counts(), train.groupby("Sex")["Survived"].mean()

(0    577
 1    314
 Name: Sex, dtype: int64,
 Sex
 0    0.188908
 1    0.742038
 Name: Survived, dtype: float64)

In [2910]:
def age_encoder(age):
    if age <= 18:
        return 0
    elif age <= 30:
        return 1
    elif age <= 50:
        return 2
    elif age > 50:
        return 3

In [2911]:
train["Age"] = train["Age"].apply(age_encoder)
test["Age"] = test["Age"].apply(age_encoder)

In [2912]:
train["Age"].value_counts(), train.groupby("Age")["Survived"].mean()

(1.0    270
 2.0    241
 0.0    139
 3.0     64
 Name: Age, dtype: int64,
 Age
 0.0    0.503597
 1.0    0.355556
 2.0    0.423237
 3.0    0.343750
 Name: Survived, dtype: float64)

In [2913]:
train["Family"] = train["SibSp"] + train["Parch"]
test["Family"] = test["SibSp"] + test["Parch"]
train["Alone"] = train["Family"].apply(lambda x: 1 if x==0 else 0)
test["Alone"] = test["Family"].apply(lambda x: 1 if x==0 else 0)

In [2914]:
train["Family"].value_counts(), train.groupby("Family")["Survived"].mean()

(0     537
 1     161
 2     102
 3      29
 5      22
 4      15
 6      12
 10      7
 7       6
 Name: Family, dtype: int64,
 Family
 0     0.303538
 1     0.552795
 2     0.578431
 3     0.724138
 4     0.200000
 5     0.136364
 6     0.333333
 7     0.000000
 10    0.000000
 Name: Survived, dtype: float64)

In [2915]:
def family_encoder(family):
    if family == 0:
        return 0
    elif family == 1:
        return 1
    elif family == 2:
        return 2
    else:
        return 3

In [2916]:
train["Family"] = train["Family"].apply(family_encoder)
test["Family"] = test["Family"].apply(family_encoder)

In [2917]:
train["Family"].value_counts(), train.groupby("Family")["Survived"].mean()

(0    537
 1    161
 2    102
 3     91
 Name: Family, dtype: int64,
 Family
 0    0.303538
 1    0.552795
 2    0.578431
 3    0.340659
 Name: Survived, dtype: float64)

In [2918]:
train["Alone"].value_counts(), train.groupby("Alone")["Survived"].mean()

(1    537
 0    354
 Name: Alone, dtype: int64,
 Alone
 0    0.505650
 1    0.303538
 Name: Survived, dtype: float64)

In [2919]:
train["Cabin"].str[0].value_counts()

C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: Cabin, dtype: int64

In [2920]:
train["Cabin"] = train["Cabin"].str[0]
test["Cabin"] = test["Cabin"].str[0]

In [2921]:
def cabin_encoder(cabin):
    if cabin == "B":
        return 0
    elif cabin == "C":
        return 1
    elif cabin == "D":
        return 2
    elif cabin == "E":
        return 3
    else:
        return 4

In [2922]:
train["Cabin"] = train["Cabin"].apply(cabin_encoder)
test["Cabin"] = test["Cabin"].apply(cabin_encoder)

In [2923]:
train["Cabin"].value_counts(), train.groupby("Cabin")["Survived"].mean()

(4    720
 1     59
 0     47
 2     33
 3     32
 Name: Cabin, dtype: int64,
 Cabin
 0    0.744681
 1    0.593220
 2    0.757576
 3    0.750000
 4    0.309722
 Name: Survived, dtype: float64)

In [2924]:
def age_encoder(fare):
    if fare <= 8:
        return 0
    elif fare <= 15:
        return 1
    elif fare <= 31:
        return 2
    elif fare > 31:
        return 3

In [2925]:
train["Fare"] = train["Fare"].apply(age_encoder)
test["Fare"] = test["Fare"].apply(age_encoder)

In [2926]:
train["Fare"].value_counts(), train.groupby("Fare")["Survived"].mean()

(0    241
 3    222
 1    217
 2    211
 Name: Fare, dtype: int64,
 Fare
 0    0.215768
 1    0.285714
 2    0.469194
 3    0.581081
 Name: Survived, dtype: float64)

In [2927]:
train.groupby("Embarked")["Survived"].mean()

Embarked
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64

In [2928]:
def embarked_encoder(embark):
    if embark == "S":
        return 0
    elif embark == "Q":
        return 1
    elif embark == "C":
        return 2

In [2929]:
train["Embarked"] = train["Embarked"].apply(embarked_encoder)
test["Embarked"] = test["Embarked"].apply(embarked_encoder)

In [2930]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Family,Alone
0,1,0,3,"Braund, Mr. Owen Harris",0,1.0,1,0,A/5 21171,0,4,0.0,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,2.0,1,0,PC 17599,3,1,2.0,2,1,0
2,3,1,3,"Heikkinen, Miss. Laina",1,1.0,0,0,STON/O2. 3101282,0,4,0.0,1,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,2.0,1,0,113803,3,1,0.0,2,1,0
4,5,0,3,"Allen, Mr. William Henry",0,2.0,0,0,373450,1,4,0.0,0,0,1


# **Prepare Train and Test data**

In [2932]:
train.drop(columns=["PassengerId", "Name", "SibSp", "Parch", "Ticket"], inplace=True)
test.drop(columns=["PassengerId", "Name", "SibSp", "Parch", "Ticket"], inplace=True)

In [2933]:
train.head()

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Family,Alone
0,0,3,0,1.0,0,4,0.0,0,1,0
1,1,1,1,2.0,3,1,2.0,2,1,0
2,1,3,1,1.0,0,4,0.0,1,0,1
3,1,1,1,2.0,3,1,0.0,2,1,0
4,0,3,0,2.0,1,4,0.0,0,0,1


In [2935]:
test.head()

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Family,Alone
0,3,0,2.0,0.0,4,1,0,0,1
1,3,1,2.0,0.0,4,0,2,1,0
2,2,0,3.0,1.0,4,1,0,0,1
3,3,0,1.0,1.0,4,0,0,0,1
4,3,1,1.0,1.0,4,0,2,2,0


In [2936]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       714 non-null    float64
 4   Fare      891 non-null    int64  
 5   Cabin     891 non-null    int64  
 6   Embarked  889 non-null    float64
 7   Title     891 non-null    int64  
 8   Family    891 non-null    int64  
 9   Alone     891 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 69.7 KB


In [2937]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       332 non-null    float64
 3   Fare      417 non-null    float64
 4   Cabin     418 non-null    int64  
 5   Embarked  418 non-null    int64  
 6   Title     418 non-null    int64  
 7   Family    418 non-null    int64  
 8   Alone     418 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 29.5 KB


In [2938]:
X = train.drop(columns="Survived")
y = train["Survived"]

In [2939]:
X

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Family,Alone
0,3,0,1.0,0,4,0.0,0,1,0
1,1,1,2.0,3,1,2.0,2,1,0
2,3,1,1.0,0,4,0.0,1,0,1
3,1,1,2.0,3,1,0.0,2,1,0
4,3,0,2.0,1,4,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...
886,2,0,1.0,1,4,0.0,4,0,1
887,1,1,1.0,2,0,0.0,1,0,1
888,3,1,NaN,2,4,0.0,1,3,0
889,1,0,1.0,2,1,2.0,0,0,1


In [2940]:
test

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Family,Alone
0,3,0,2.0,0.0,4,1,0,0,1
1,3,1,2.0,0.0,4,0,2,1,0
2,2,0,3.0,1.0,4,1,0,0,1
3,3,0,1.0,1.0,4,0,0,0,1
4,3,1,1.0,1.0,4,0,2,2,0
...,...,...,...,...,...,...,...,...,...
413,3,0,NaN,1.0,4,0,0,0,1
414,1,1,2.0,3.0,1,2,4,0,1
415,3,0,2.0,0.0,4,0,0,0,1
416,3,0,NaN,1.0,4,0,0,0,1


In [2941]:
# Impute missing values with KNN Imputer
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
X = imputer.fit_transform(X)
test = imputer.transform(test)

# **Model Building and training**

In [2942]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [2943]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train = torch.from_numpy(X_train).to(torch.float32)
X_test = torch.from_numpy(X_test).to(torch.float32)
y_train = torch.from_numpy(np.array(y_train)).to(torch.float64)
y_test = torch.from_numpy(np.array(y_test)).to(torch.float64)

In [2944]:
X_test.shape, y_test.shape

(torch.Size([179, 9]), torch.Size([179]))

In [2945]:
train_ds = TensorDataset(X_train, y_train)

In [2946]:
test_ds = TensorDataset(X_test, y_test)

In [2947]:
train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=8, shuffle=False)

In [2948]:
class SimpleModel(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape):
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(in_features=input_shape, out_features=hidden_units),
                                    nn.ReLU(),
                                    nn.Linear(in_features=hidden_units, out_features=output_shape))
    def forward(self, x):
        return self.layers(x)

In [2949]:
simple_model = SimpleModel(input_shape=9, hidden_units=8, output_shape=1)
simple_model

SimpleModel(
  (layers): Sequential(
    (0): Linear(in_features=9, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [2950]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=simple_model.parameters(), lr=0.001)

In [2951]:
from tqdm.auto import tqdm

In [2952]:
epochs = 51
train_losses = []
train_accs = []
test_losses = []
test_accs = []

for epoch in tqdm(range(epochs)):
    train_loss = 0
    train_acc = 0
    for X, y in train_dl:
        
        simple_model.train()
        
        y_logits = simple_model(X).squeeze()
        y_pred = torch.round(torch.sigmoid(y_logits))
        
        loss = loss_fn(y_logits, y)
        train_loss += loss.item()
        acc = torch.sum((y_pred == y)) / len(X)
        train_acc += acc.item()
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
    train_acc = train_acc/len(train_dl)
    train_loss = train_loss/len(train_dl)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    if epoch%10 == 0:
        print(f"====================================\nEpoch: {epoch}")
        print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f}")
    test_loss = 0
    test_acc = 0    
    for X, y in test_dl:
        simple_model.eval()
        with torch.inference_mode():
            y_logits = simple_model(X).squeeze()
            y_pred = torch.round(torch.sigmoid(y_logits))
        
            loss = loss_fn(y_logits, y)
            test_loss += loss.item()
            acc = torch.sum((y_pred == y)) / len(X)
            test_acc += acc.item()
            
    test_acc = test_acc/len(test_dl)
    test_loss = test_loss/len(test_dl)
    test_losses.append(train_loss)
    test_accs.append(train_acc)            
    if epoch%10 == 0:
        print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")    

  0%|          | 0/51 [00:00<?, ?it/s]

Epoch: 0
Train Loss: 0.6249 | Train Accuracy: 0.6489
Test Loss: 0.5938 | Test Accuracy: 0.6322
Epoch: 10
Train Loss: 0.4741 | Train Accuracy: 0.7935
Test Loss: 0.4370 | Test Accuracy: 0.8225
Epoch: 20
Train Loss: 0.4588 | Train Accuracy: 0.7963
Test Loss: 0.4234 | Test Accuracy: 0.8496
Epoch: 30
Train Loss: 0.4508 | Train Accuracy: 0.8104
Test Loss: 0.4187 | Test Accuracy: 0.8442
Epoch: 40
Train Loss: 0.4444 | Train Accuracy: 0.8076
Test Loss: 0.4148 | Test Accuracy: 0.8442
Epoch: 50
Train Loss: 0.4418 | Train Accuracy: 0.8216
Test Loss: 0.4110 | Test Accuracy: 0.8442


In [2953]:
class DNN(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape):
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(in_features=input_shape, out_features=hidden_units),
                                    nn.ReLU(),
                                    nn.Linear(in_features=hidden_units, out_features=hidden_units),
                                    nn.ReLU(),
                                    nn.Linear(in_features=hidden_units, out_features=output_shape))
    def forward(self, x):
        return self.layers(x)

In [2954]:
dnn = DNN(9, 8, 1)
dnn

DNN(
  (layers): Sequential(
    (0): Linear(in_features=9, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [2955]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=dnn.parameters(), lr=0.001)

In [2956]:
epochs = 151
train_losses = []
train_accs = []
test_losses = []
test_accs = []

for epoch in tqdm(range(epochs)):
    train_loss = 0
    train_acc = 0
    for X, y in train_dl:
        
        dnn.train()
        
        y_logits = dnn(X).squeeze()
        y_pred = torch.round(torch.sigmoid(y_logits))
        
        loss = loss_fn(y_logits, y)
        train_loss += loss.item()
        acc = torch.sum((y_pred == y)) / len(X)
        train_acc += acc.item()
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
    train_acc = train_acc/len(train_dl)
    train_loss = train_loss/len(train_dl)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    if epoch%10 == 0:
        print(f"====================================\nEpoch: {epoch}")
        print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f}")
    test_loss = 0
    test_acc = 0    
    for X, y in test_dl:
        dnn.eval()
        with torch.inference_mode():
            y_logits = dnn(X).squeeze()
            y_pred = torch.round(torch.sigmoid(y_logits) )
        
            loss = loss_fn(y_logits, y)
            test_loss += loss.item()
            acc = torch.sum((y_pred == y)) / len(X)
            test_acc += acc.item()
            
    test_acc = test_acc/len(test_dl)
    test_loss = test_loss/len(test_dl)
    test_losses.append(train_loss)
    test_accs.append(train_acc)            
    if epoch%10 == 0:
        print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")    

  0%|          | 0/151 [00:00<?, ?it/s]

Epoch: 0
Train Loss: 0.6507 | Train Accuracy: 0.6657
Test Loss: 0.6142 | Test Accuracy: 0.6812
Epoch: 10
Train Loss: 0.4594 | Train Accuracy: 0.8062
Test Loss: 0.4106 | Test Accuracy: 0.8442
Epoch: 20
Train Loss: 0.4387 | Train Accuracy: 0.8104
Test Loss: 0.4030 | Test Accuracy: 0.8496
Epoch: 30
Train Loss: 0.4304 | Train Accuracy: 0.8160
Test Loss: 0.4006 | Test Accuracy: 0.8496
Epoch: 40
Train Loss: 0.4268 | Train Accuracy: 0.8202
Test Loss: 0.4026 | Test Accuracy: 0.8442
Epoch: 50
Train Loss: 0.4224 | Train Accuracy: 0.8118
Test Loss: 0.4016 | Test Accuracy: 0.8496
Epoch: 60
Train Loss: 0.4151 | Train Accuracy: 0.8272
Test Loss: 0.4073 | Test Accuracy: 0.8551
Epoch: 70
Train Loss: 0.4149 | Train Accuracy: 0.8287
Test Loss: 0.4052 | Test Accuracy: 0.8551
Epoch: 80
Train Loss: 0.4130 | Train Accuracy: 0.8287
Test Loss: 0.4054 | Test Accuracy: 0.8551
Epoch: 90
Train Loss: 0.4103 | Train Accuracy: 0.8230
Test Loss: 0.4094 | Test Accuracy: 0.8551
Epoch: 100
Train Loss: 0.4077 | Train Acc

In [2957]:
test = torch.from_numpy(test).float()
test

tensor([[3.0000, 0.0000, 2.0000,  ..., 0.0000, 0.0000, 1.0000],
        [3.0000, 1.0000, 2.0000,  ..., 2.0000, 1.0000, 0.0000],
        [2.0000, 0.0000, 3.0000,  ..., 0.0000, 0.0000, 1.0000],
        ...,
        [3.0000, 0.0000, 2.0000,  ..., 0.0000, 0.0000, 1.0000],
        [3.0000, 0.0000, 1.6000,  ..., 0.0000, 0.0000, 1.0000],
        [3.0000, 0.0000, 0.2000,  ..., 3.0000, 2.0000, 0.0000]])

In [2958]:
with torch.inference_mode():
    y_logits = dnn(test).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))

In [2959]:
predictions = y_pred.numpy().astype(int)

In [2963]:
output = pd.DataFrame({'PassengerId': pd.read_csv("/kaggle/input/titanic/test.csv").PassengerId, 'Survived': predictions})
output.to_csv('submission_high_score_mh1.csv', index=False)